# Script abaixo eu costumo utilizar para meus estudos, ele configura o spark no google collab e faz várias outras coisas para facilitar meus trabalhos

In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install -q findspark pandas numpy

# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.2.2-bin-hadoop3.2')


from pyspark.sql import functions as func
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import warnings, re

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


# Instanciando Spark
spark = SparkSession.builder.master('local[*]').getOrCreate()

--2023-10-06 18:33:15--  https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301112604 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.2-bin-hadoop3.2.tgz’

spark-3.2.2-bin-had 100%[===================>] 287.16M  7.08MB/s    in 3m 26s  

2023-10-06 18:36:41 (1.39 MB/s) - ‘spark-3.2.2-bin-hadoop3.2.tgz’ saved [301112604/301112604]



# Criando 2 dataframes com o schema idêntico para realizar o union

In [5]:
df_1 = (spark.read.format("csv").options(header="true")
    .load("/content/Draft_Picks_From_Duke.csv"))

df_2 = (spark.read.format("csv").options(header="true")
    .load("/content/Draft_Picks_From_Duke.csv"))

In [7]:
df_1.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Rd: string (nullable = true)
 |-- Overall: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- G: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- FG: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- 3P: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- FT: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- ORB: string (nullable = true)
 |-- TRB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- MP/G: string (nullable = true)
 |-- PTS/G: string (nullable = true)
 |-- TRB/G: string (nullable = true)
 |-- AST/G

In [8]:
df_2.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Rd: string (nullable = true)
 |-- Overall: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- G: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- FG: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- 3P: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- FT: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- ORB: string (nullable = true)
 |-- TRB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- MP/G: string (nullable = true)
 |-- PTS/G: string (nullable = true)
 |-- TRB/G: string (nullable = true)
 |-- AST/G

# Realizando o Union "convencional" que costumammos aprender com SQL, onde o layout das nossas tabelas devem ser idênticos.

In [9]:
union_df = df_1.union(df_2)

In [15]:
union_df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Rd: string (nullable = true)
 |-- Overall: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- G: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- FG: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- 3P: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- FT: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- ORB: string (nullable = true)
 |-- TRB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- MP/G: string (nullable = true)
 |-- PTS/G: string (nullable = true)
 |-- TRB/G: string (nullable = true)
 |-- AST/G

# Obtendo uma lista das nossas colunas para facilitar meus próximos passos

In [11]:
df_2.columns

['Player',
 'Year',
 'Rd',
 'Overall',
 'From',
 'To',
 'G',
 'MP',
 'FG',
 'FGA',
 '3P',
 '3PA',
 'FT',
 'FTA',
 'ORB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'FG%',
 '3P%',
 'FT%',
 'MP/G',
 'PTS/G',
 'TRB/G',
 'AST/G']

# Criando um novo dataframe com as mesmas colunas do df_2 mas em ordem diferente, as colunas que estão na linha 1 são as que eu tirei da ordem, menos a Player

In [12]:
df_2_nova_ordem = df_2.select('Player', 'BLK', 'PTS/G', 'FGA', 'FG%', 'FT%',
 'Year',
 'Rd',
 'Overall',
 'From',
 'To',
 'G',
 'MP',
 'FG',
 '3P',
 '3PA',
 'FT',
 'FTA',
 'ORB',
 'TRB',
 'AST',
 'STL',
 'TOV',
 'PF',
 'PTS',
 '3P%',
 'MP/G',
 'TRB/G',
 'AST/G')

# Fazendo um Union by Name, ou seja, temos exatamente as mesmas colunas nos 2 dfs, mas em diferentes ordens.

In [14]:
union_df_by_name = df_1.unionByName(df_2_nova_ordem)

In [16]:
union_df_by_name.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Rd: string (nullable = true)
 |-- Overall: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- G: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- FG: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- 3P: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- FT: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- ORB: string (nullable = true)
 |-- TRB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- MP/G: string (nullable = true)
 |-- PTS/G: string (nullable = true)
 |-- TRB/G: string (nullable = true)
 |-- AST/G

# Alterando o tipo da coluna year para inteiro para testar se o union by name aceite que os tipos das colunas sejam diferentes.

In [19]:
df_2_nova_ordem = df_2_nova_ordem.withColumn("Year", F.col("Year").cast("int"))

In [18]:
df_2_nova_ordem.printSchema()

root
 |-- Player: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- PTS/G: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Rd: string (nullable = true)
 |-- Overall: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- G: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- FG: string (nullable = true)
 |-- 3P: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- FT: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- ORB: string (nullable = true)
 |-- TRB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- MP/G: string (nullable = true)
 |-- TRB/G: string (nullable = true)
 |-- AST/

In [20]:
union_df_by_name = df_1.unionByName(df_2_nova_ordem)

# Aceita mas volta a ser string como na tabela da "esquerda"

In [21]:
union_df_by_name.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Rd: string (nullable = true)
 |-- Overall: string (nullable = true)
 |-- From: string (nullable = true)
 |-- To: string (nullable = true)
 |-- G: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- FG: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- 3P: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- FT: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- ORB: string (nullable = true)
 |-- TRB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- MP/G: string (nullable = true)
 |-- PTS/G: string (nullable = true)
 |-- TRB/G: string (nullable = true)
 |-- AST/G